# Pandects API quickstart (Python)

Prerequisites:
- Run `pip install requests` to install `requests` (if not already installed)
- Obtain an API key from [pandects.org](pandects.org), exported as `PANDECTS_API_KEY` (starts with `pdcts_`)


In [1]:
import os
import re
from collections import Counter

import requests


In [2]:
# --- Config ---
API_BASE_URL = "https://api.pandects.org"
API_KEY = os.getenv("PANDECTS_API_KEY")

if not API_KEY:
    raise RuntimeError(
        "Missing PANDECTS_API_KEY. Export your key (starts with pdcts_) and re-run this cell."
    )

session = requests.Session()
session.headers.update({"Accept": "application/json", "X-API-Key": API_KEY})

API_BASE_URL


'https://api.pandects.org'

## 1) Download clause sections (`GET /v1/search`)

Each result is a clause section with metadata and XML.


In [ ]:
r = session.get(
    f"{API_BASE_URL}/v1/search",
    params={"page": 1, "pageSize": 25},
    timeout=30,
)
r.raise_for_status()
payload = r.json()

sections = payload["results"]
print("Returned sections:", len(sections))
print("Access tier:", payload.get("access", {}).get("tier"))

example = sections[0]
{k: example[k] for k in ["year", "acquirer", "target", "articleTitle", "sectionTitle"]}


Returned sections: 25
Access tier: api_key


{'year': 2008,
 'acquirer': 'InBev SA',
 'target': 'Anheuser-Busch Cos., Inc.',
 'articleTitle': 'ARTICLE IX Miscellaneous and General',
 'sectionTitle': '9.10. Transfer Taxes.'}

In [4]:
# Fun, small thing: quick counts on this sample
print("Top acquirers:")
for name, n in Counter(s["acquirer"] for s in sections).most_common(5):
    print(f"- {name} ({n})")

print("\nYears:")
for year, n in Counter(s["year"] for s in sections).most_common(5):
    print(f"- {year} ({n})")


Top acquirers:
- United Technologies Corp. (4)
- Energy Transfer Equity LP (2)
- Broadcom, Inc. (2)
- InBev SA (1)
- Leidos Holdings, Inc. (1)

Years:
- 2017 (5)
- 2019 (5)
- 2008 (3)
- 2016 (3)
- 2018 (2)


## 2) Quick XML preview (no extra packages)

The API returns XML. This turns it into a quick, readable text preview.


In [5]:
TAG_RE = re.compile(r"<[^>]+>")

def xml_to_preview_text(xml: str, *, max_chars: int = 700) -> str:
    text = TAG_RE.sub(" ", xml)
    text = re.sub(r"\s+", " ", text).strip()
    return text if len(text) <= max_chars else text[:max_chars].rstrip() + "…"

print(example["year"], "|", example["acquirer"], "→", example["target"])
print(example["articleTitle"], "/", example["sectionTitle"])
print("\nPreview:\n")
print(xml_to_preview_text(example["xml"]))


2008 | InBev SA → Anheuser-Busch Cos., Inc.
ARTICLE IX Miscellaneous and General / 9.10. Transfer Taxes.

Preview:

All transfer, documentary, sales, use, stamp, registration and other such Taxes and fees (including penalties and interest) incurred in connection with the Merger shall be paid by Parent and Merger Sub when due, and Parent and Merger Sub will indemnify the Company against liability for any such taxes.


## 3) Download the full agreement XML (`GET /v1/agreements/<uuid>`)

Use a section’s `agreementUuid` to fetch the entire agreement.


In [ ]:
agreement_uuid = example["agreementUuid"]

r = session.get(f"{API_BASE_URL}/v1/agreements/{agreement_uuid}", timeout=30)
r.raise_for_status()
agreement = r.json()

print("Response keys:", sorted(agreement.keys()))
print("XML chars:", len(agreement["xml"]))
print("\nAgreement preview:\n")
print(xml_to_preview_text(agreement["xml"], max_chars=900))


Response keys: ['acquirer', 'target', 'url', 'xml', 'year']
XML chars: 221653

Agreement preview:

InBev SA Anheuser-Busch Cos., Inc. 2008-06-11 https://www.sec.gov/Archives/edgar/data/310569/000119312508151934/dex21.htm html AGREEMENT AND PLAN OF MERGER AGREEMENT AND PLAN OF MERGER (hereinafter called this “Agreement“), dated as of July 13, 2008, by and among Anheuser-Busch Companies, Inc., a Delaware corporation (the “Company“), InBev N.V./S.A., a public company organized under the laws of Belgium (“Parent“), and Pestalozzi Acquisition Corp., a Delaware corporation and a wholly owned subsidiary of Parent (“Merger Sub,” the Company and Merger Sub sometimes being hereinafter collectively referred to as the “Constituent Corporations“). RECITALS WHEREAS, the respective boards of directors of each of Parent, Merger Sub and the Company have approved the merger of Merger Sub with and into the Company (the “Merger“) upon the terms and subject to the conditions set forth in this Agreement an…